<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/%D0%93%D0%BB%D0%B0%D0%B4%D1%8B%D1%88%D0%B5%D0%B2%D0%92%D0%92_NLP_HW_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 7 Гладышев ВВ

Берем отызывы за лето

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel("отзывы за лето.xls")

In [ ]:
data.tail(3)

,Rating,Content,Date
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [ ]:
from datetime import datetime

In [ ]:
type(data["Date"].values[0])

str

In [ ]:
datetime_object = datetime.strptime(data["Date"].values[0], '%Y-%m-%d').date()
datetime_object

datetime.date(2017, 8, 14)

In [ ]:
data["Year"] = [datetime.strptime(dt, '%Y-%m-%d').year for dt in data["Date"].values]
data["Month"] = [datetime.strptime(dt, '%Y-%m-%d').month for dt in data["Date"].values]
data["Day"] = [datetime.strptime(dt, '%Y-%m-%d').day for dt in data["Date"].values]
data.head()

,Rating,Content,Date,Year,Month,Day
0,5,It just works!,2017-08-14,2017,8,14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,2017,8,14
2,5,Отлично все,2017-08-14,2017,8,14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,2017,8,14
4,5,"Очень удобно, работает быстро.",2017-08-14,2017,8,14


1. Учим conv сеть для классификации

In [ ]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=1aa041985bb7cdcc6fc2967078be104966db45af7b2c4c5e3bac536bfb0f231b
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 907.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=1815c6b64cdcaa2df4710fda907f8712be66c98413f6db1e814ccaeba3e9c5ec
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [ ]:
# for word in words:
#     print(word, [p.normal_form for p in morph.parse(word)])

In [ ]:
# sw = set(get_stop_words("ru"))
# exclude = set(punctuation)
# morpher = MorphAnalyzer()

# def preprocess_text(txt):
#     txt = str(txt)
#     txt = "".join(c for c in txt if c not in exclude)
#     txt = txt.lower()
#     txt = re.sub("\sне", "не", txt)
#     txt = [str(wd.normal_form) for word in txt.split() for wd in morpher.parse(word) if word not in sw]   # txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
#     return " ".join(txt)

In [ ]:
data['Content_prep'] = data['Content'].apply(preprocess_text)
data.tail()

,Rating,Content,Date,Year,Month,Day,Content_prep
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,2017,6,1,шляпас роот право бесполезный прогаразрабыв ох...
20655,5,Ок,2017-06-01,2017,6,1,около
20656,4,Доволен,2017-06-01,2017,6,1,довольный
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,2017,6,1,песопаснастя рута нужын
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01,2017,6,1,сбербанк бомбовый компания сегодняшний лидер б...


In [ ]:
train_corpus = " ".join(data['Content_prep'])

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens_tot = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokens_filtered = [word for word in tokens_tot if word.isalnum()]

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)

In [ ]:
vocabulary = {v: i+100  for i, v in enumerate(dist) if dist[v]>1}

In [ ]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
data_embeded = np.asarray([text_to_sequence(text, 8) for text in data['Content_prep']], dtype=np.int32)
data_embeded = np.hstack((data_embeded, (data['Month'].values+50).reshape(data['Month'].values.shape[0], 1)))
data_embeded = np.hstack((data_embeded, (data['Day'].values).reshape(data['Day'].values.shape[0], 1)))
data_embeded

array([[   0,    0,    0, ...,  102,   58,   14],
       [ 107,  108,  109, ...,  115,   58,   14],
       [   0,    0,    0, ...,  116,   58,   14],
       ...,
       [   0,    0,    0, ...,  274,   56,    1],
       [   0,    0,    0, ...,  525,   56,    1],
       [   0,  182, 9232, ..., 3814,   56,    1]])

In [ ]:
np.max(data_embeded)

12924

In [ ]:
# data_embeded = data_embeded / np.max(data_embeded)
# data_embeded

In [ ]:
data_embeded[-1]

array([   0,  182, 9232, 4543, 4368, 6421,  427, 3814,   56,    1])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data_embeded, y, stratify=y, random_state=43)

In [ ]:
train_x.shape

(15494, 10)

In [ ]:
tr_data = train_x.reshape(train_x.shape[0], train_x.shape[1], 1)
vd_data = valid_x.reshape(valid_x.shape[0], valid_x.shape[1], 1)
tr_data.shape

(15494, 10, 1)

In [ ]:
def to_one_hot(Y):
    n_col = np.amax(Y)
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]-1] = 1.
    return binarized

In [ ]:
train_y_o = to_one_hot(train_y)
valid_y_o = to_one_hot(valid_y)
train_y_o

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

### Строим свёрточную нейронную сеть

In [ ]:
import tensorflow as tf

# !pip install -q tensorflow-hub
# !pip install -q tensorflow-datasets
import tensorflow_hub as hub
# import tensorflow_datasets as tfds

import keras

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten
from keras.utils import to_categorical

from typing import List, Optional, Dict, Tuple

In [ ]:
def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [ ]:
def net_gen(lay_cnt=1, filters=32, neurons=32):
    # конфигурирование слоев нейросети
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv1D(filters, 3, padding='same', activation='sigmoid', input_shape=(tr_data.shape[1], 1)))   # relu
    # model.add(Conv1D(filters, 3, activation='relu'))
    # model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    if lay_cnt == 2:
        model.add(Conv1D(filters, 3, padding='same', activation='sigmoid')) # relu
        # model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(neurons, activation='sigmoid'))   # relu
    # model.add(Dense(neurons, activation='relu'))))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax')) # sigmoid'))

    # компиляция модели
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.AUC()])

    return model

In [ ]:
def net_iter(model, ep=10, batch_size=256):
    history = model.fit(tr_data,
                        train_y_o,
                        batch_size=batch_size,
                        epochs=ep,
                        validation_data=(vd_data, valid_y_o),
                        shuffle=True,
                        verbose=0)
    return history.history

In [ ]:
summary_data = pd.DataFrame(data=[[0, 0, 0, 0, 0, 0, 0, 0]],
                            columns=['layers', 'filters', 'neurons', 'mean_train_score', 'mean_val_score', 'variance', 'l_b', 'r_b'])
cnt = 0
f_n = [128, 256]
l_c = [1, 2]
nr = [32, 64]
epochs = 15

# net_res = []

for l_c_i in l_c:
    for f_n_i in f_n:
        for nr_i in nr:
            err_score = []
            err_val_score = []
            print(f"Модель: слоёв {l_c_i}, фильтров {f_n_i}, нейронов {nr_i}, эпох {epochs}\n")
            model = net_gen(filters=f_n_i, lay_cnt=l_c_i, neurons=nr_i)
            model.summary()
            for i in range(5):
                print(f"\tИтерация: {i+1}")
                hist = net_iter(model, ep=epochs)
                # net_res.append(hist)
                err_score.append(hist[list(hist.keys())[1]][-1])
                err_val_score.append(hist[list(hist.keys())[3]][-1])

            vr = np.var(err_val_score)
            l_b, r_b = calculate_confidence_interval(err_val_score)
            mean_err_score, mean_err_val_score = np.mean(err_score), np.mean(err_val_score)
            print(f"Ошибки на train. Средняя: {mean_err_score} список: {err_score}")
            print(f"Ошибки на valid. Средняя: {mean_err_val_score} список: {err_val_score}")
            print(f"Дисперсия ошибки на valid: {vr}")
            print(f"Доверительный интервал: {l_b} - {r_b}\n\n")
            summary_data.loc[cnt, ['layers', 'filters', 'neurons', 'mean_train_score',
                                    'mean_val_score', 'variance', 'l_b', 'r_b']] = [l_c_i, f_n_i,
                                    nr_i, np.mean(err_score), np.mean(err_val_score), vr, l_b, r_b]
            cnt += 1

Модель: слоёв 1, фильтров 128, нейронов 32, эпох 15

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_18 (Conv1D)           (None, 10, 128)           512       
_________________________________________________________________
dropout_13 (Dropout)         (None, 10, 128)           0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                40992     
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 5)                 165       
Total params: 41,669
Trainable params: 41,669
Non-trainable params:

In [ ]:
summary_data

,layers,filters,neurons,mean_train_score,mean_val_score,variance,l_b,r_b
0,1.0,128.0,32.0,0.815993,0.816312,0.00000,0.816312,0.816312
1,1.0,128.0,64.0,0.816304,0.816312,0.00000,0.816312,0.816312
2,1.0,256.0,32.0,0.856316,0.854782,0.00002,0.851025,0.862458
3,1.0,256.0,64.0,0.816277,0.816312,0.00000,0.816312,0.816312
4,2.0,128.0,32.0,0.815799,0.816312,0.00000,0.816312,0.816312
5,2.0,128.0,64.0,0.816246,0.816312,0.00000,0.816312,0.816312
6,2.0,256.0,32.0,0.816176,0.816312,0.00000,0.816312,0.816312
7,2.0,256.0,64.0,0.816258,0.816312,0.00000,0.816312,0.816312


In [ ]:
summary_data.sort_values('mean_val_score', ascending=False)

,layers,filters,neurons,mean_train_score,mean_val_score,variance,l_b,r_b
2,1.0,256.0,32.0,0.856316,0.854782,0.00002,0.851025,0.862458
0,1.0,128.0,32.0,0.815993,0.816312,0.00000,0.816312,0.816312
1,1.0,128.0,64.0,0.816304,0.816312,0.00000,0.816312,0.816312
3,1.0,256.0,64.0,0.816277,0.816312,0.00000,0.816312,0.816312
4,2.0,128.0,32.0,0.815799,0.816312,0.00000,0.816312,0.816312
5,2.0,128.0,64.0,0.816246,0.816312,0.00000,0.816312,0.816312
6,2.0,256.0,32.0,0.816176,0.816312,0.00000,0.816312,0.816312
7,2.0,256.0,64.0,0.816258,0.816312,0.00000,0.816312,0.816312


In [ ]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv1D(128, 3, padding='same', activation='sigmoid', input_shape=(tr_data.shape[1], 1))) # sigmoid'))relu
model.add(Conv1D(128, 3, padding='same', activation='sigmoid'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))
# if lay_cnt == 2:
model.add(Conv1D(128, 3, padding='same', activation='sigmoid'))
model.add(Conv1D(128, 3, padding='same', activation='sigmoid'))
model.add(MaxPooling1D(pool_size=2))
#     model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(32, activation='sigmoid')) # 'relu'))
# model.add(Dense(neurons, activation='relu'))))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax')) # sigmoid'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.AUC()])

In [ ]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_30 (Conv1D)           (None, 10, 128)           512       
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 10, 128)           49280     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 5, 128)            0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 5, 128)            49280     
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 5, 128)            49280     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 2, 128)            0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 256)             

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback],
                 shuffle=True,
                 verbose=1)

Epoch 1/1000
31/31 [==============================] - 0s 10ms/step - loss: 0.7934 - auc_17: 0.9171 - val_loss: 0.7814 - val_auc_17: 0.9195
Epoch 2/1000
31/31 [==============================] - 0s 8ms/step - loss: 0.7922 - auc_17: 0.9177 - val_loss: 0.7826 - val_auc_17: 0.9195
Epoch 3/1000
31/31 [==============================] - 0s 7ms/step - loss: 0.7907 - auc_17: 0.9176 - val_loss: 0.7798 - val_auc_17: 0.9196
Epoch 4/1000
31/31 [==============================] - 0s 7ms/step - loss: 0.7915 - auc_17: 0.9178 - val_loss: 0.7819 - val_auc_17: 0.9196
Epoch 5/1000
31/31 [==============================] - 0s 7ms/step - loss: 0.7922 - auc_17: 0.9172 - val_loss: 0.7874 - val_auc_17: 0.9185
Epoch 6/1000
31/31 [==============================] - 0s 7ms/step - loss: 0.7942 - auc_17: 0.9172 - val_loss: 0.7804 - val_auc_17: 0.9199


2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

## Попробуем word2vec с LogisticRegression

In [ ]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score

from tqdm import tqdm

In [ ]:
modelW2V = Word2Vec(sentences=data['Content_prep'].apply(str.split), size=100, window=5, min_count=1)

In [ ]:
# modelW2V.wv.vocab

In [ ]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(100)
    n_w2v = 0
    for wrd in txt:
        if wrd in modelW2V:
            vector_w2v += modelW2V[wrd]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [ ]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data['Content_prep'].values, y, stratify=y, random_state=43)

In [ ]:
arr_vect = []
for txt in tqdm(train_x):
    arr_vect.append(get_vect_mean(txt))

arr_vect_valid = []
for txt in tqdm(valid_x):
    arr_vect_valid.append(get_vect_mean(txt))

train_w2v = np.asarray(arr_vect)
valid_w2v = np.asarray(arr_vect_valid)

  0%|          | 0/15494 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
100%|██████████| 5165/5165 [00:01<00:00, 3163.71it/s]


In [ ]:
lgr_w2v = LogisticRegression()

In [ ]:
lgr_w2v.fit(train_w2v, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = lgr_w2v.predict_proba(valid_w2v)

In [ ]:
y_pred

array([[0.10185659, 0.03429833, 0.04154197, 0.09614552, 0.72615758],
       [0.10133931, 0.03394707, 0.04182292, 0.10397162, 0.71891908],
       [0.08912818, 0.03033862, 0.03831425, 0.10778174, 0.73443721],
       ...,
       [0.10122598, 0.03410824, 0.04165331, 0.09963014, 0.72338234],
       [0.08912818, 0.03033862, 0.03831425, 0.10778174, 0.73443721],
       [0.12001547, 0.0391226 , 0.04716077, 0.10432012, 0.68938104]])

In [ ]:
roc_auc_score(to_one_hot(valid_y), y_pred)

0.6668526427592502

## Используем word2vec в CNN

In [ ]:
tr_data = train_w2v.reshape(train_w2v.shape[0], train_w2v.shape[1], 1)
vd_data = valid_w2v.reshape(valid_w2v.shape[0], valid_w2v.shape[1], 1)
tr_data.shape

(15494, 100, 1)

In [ ]:
train_y_o = to_one_hot(train_y)
valid_y_o = to_one_hot(valid_y)
train_y_o

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [ ]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv1D(256, 10, padding='same', activation='relu', input_shape=(tr_data.shape[1], 1))) # sigmoid'))relu
model.add(Conv1D(256, 10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
# if lay_cnt == 2:
# model.add(Conv1D(256, 10, padding='same', activation='relu'))
# model.add(Conv1D(256, 10, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(64, activation='relu')) # 'relu'))
# model.add(Dense(neurons, activation='relu'))))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax')) # sigmoid'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.AUC()])

In [ ]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_46 (Conv1D)           (None, 100, 256)          2816      
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 100, 256)          655616    
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 50, 256)           0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 50, 256)           0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 12800)             0         
_________________________________________________________________
dense_48 (Dense)             (None, 64)                819264    
_________________________________________________________________
dropout_47 (Dropout)         (None, 64)              

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=512,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback],
                 shuffle=True,
                 verbose=1)

Epoch 1/1000
31/31 [==============================] - 2s 61ms/step - loss: 1.1413 - auc_24: 0.8279 - val_loss: 0.9897 - val_auc_24: 0.8518
Epoch 2/1000
31/31 [==============================] - 2s 51ms/step - loss: 1.0311 - auc_24: 0.8447 - val_loss: 0.9875 - val_auc_24: 0.8543
Epoch 3/1000
31/31 [==============================] - 2s 52ms/step - loss: 1.0305 - auc_24: 0.8443 - val_loss: 0.9788 - val_auc_24: 0.8559
Epoch 4/1000
31/31 [==============================] - 2s 51ms/step - loss: 1.0156 - auc_24: 0.8484 - val_loss: 0.9716 - val_auc_24: 0.8568
Epoch 5/1000
31/31 [==============================] - 2s 52ms/step - loss: 1.0029 - auc_24: 0.8523 - val_loss: 0.9703 - val_auc_24: 0.8594
Epoch 6/1000
31/31 [==============================] - 2s 52ms/step - loss: 0.9949 - auc_24: 0.8548 - val_loss: 0.9784 - val_auc_24: 0.8602
Epoch 7/1000
31/31 [==============================] - 2s 52ms/step - loss: 0.9974 - auc_24: 0.8540 - val_loss: 0.9631 - val_auc_24: 0.8697
Epoch 8/1000
31/31 [=======